In [9]:
import os
import dipy as dp
import nibabel as nib
import numpy as np
from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table, reorient_bvecs, GradientTable
from bonndit.shore import ShoreModel, ShoreFit

os.chdir("/home/olivier/Devel/test/samuel/in4michi/")


# Load fractional anisotropy
dti_fa = nib.load("dti_FA.nii.gz")

# Load DTI mask
dti_mask = nib.load("mask.nii.gz")

# Load and adjust tissue segmentation masks
csf_mask = nib.load("fast_pve_0.nii.gz")
gm_mask = nib.load("fast_pve_1.nii.gz")
wm_mask = nib.load("fast_pve_2.nii.gz")

dti_vecs = nib.load("dti_V1.nii.gz")

data = nib.load("complete_data/data.nii")

bvals, bvecs = read_bvals_bvecs("bvals", "bvecs")
gtab = gradient_table(bvals, bvecs)

In [7]:
model = ShoreModel(gtab)
fit = model.fit(data, wm_mask, gm_mask, csf_mask, dti_mask, 
                    dti_fa, dti_vecs, verbose=True)

100%|██████████| 460800/460800 [00:31<00:00, 14409.28it/s] 


In [8]:
fit.old_save('/home/olivier/Devel/test/samuel/in4michi/bonndit_result/')
fit.save("/home/olivier/Devel/test/samuel/in4michi/bonndit_result/"+"fit.pkl")

In [10]:
fit = ShoreFit.load("/home/olivier/Devel/test/samuel/in4michi/bonndit_result/"+"fit.pkl")

In [13]:
out, wmout, gmout, csfout = fit.fodf("data.nii", verbose=True)


Optimization:  19%|█▊        | 5/27 [00:00<00:00, 48.56it/s]

Condition number of M: inf



Optimization:   0%|          | 1140/460800 [00:30<3:21:51, 37.95it/s]

In [20]:
wmout


array([[[0.0048819 , 0.00600839, 0.00557819],
        [0.00522271, 0.00509482, 0.0057228 ],
        [0.00539661, 0.00519445, 0.00438422]],

       [[0.00495465, 0.00541302, 0.00497942],
        [0.00514935, 0.00566533, 0.00546024],
        [0.00507425, 0.00520792, 0.00552798]],

       [[0.00483064, 0.00503548, 0.00524289],
        [0.00481169, 0.00466158, 0.00497798],
        [0.00518401, 0.00469915, 0.00531369]]])

In [8]:
img = nib.Nifti1Image(out, data.affine)
nib.save(img, '/home/olivier/Devel/test/samuel/in4michi/bonndit_result/odf.nii.gz')

In [14]:
from bonndit.shore import gtab_reorient, gtab_rotate
from dipy.core.gradients import GradientTable
import numpy as np
GRADIENTS = np.array([[2500,0,0],
                      [0,2500,0],
                      [0,0,2500],
                      [0,0,0]])
gtab = GradientTable(GRADIENTS)
old_vec = np.array((0, 0, 1))
new_vec = np.array((1, 0, 0))
gtab_reorient(gtab, old_vec, new_vec).gradients

array([[    0.,     0., -2500.],
       [    0.,  2500.,     0.],
       [ 2500.,     0.,     0.],
       [    0.,     0.,     0.]])